In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_classic.embeddings import CacheBackedEmbeddings
from langchain_classic.storage import LocalFileStore
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_classic.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder

llm = ChatOpenAI(
    model="gpt-4o-mini",
)

cache_dir = LocalFileStore(root_path="./.cache/")

splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(    
    chunk_size=5000,
    chunk_overlap=1000,
)

loader = TextLoader("document.txt")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    underlying_embeddings=embeddings,
    document_embedding_cache=cache_dir,
)

vectorstore = FAISS.from_documents(
    documents=docs, 
    embedding=cached_embeddings,
)

retriever = vectorstore.as_retriever()

memory = ConversationBufferMemory(return_messages=True)

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            You are a helpful assistant. 
        
            You may also use conversation history to remember user preferences or personal details.
        
            When answering knowledge questions about the document, use ONLY the following context.
        
            If you don't know the answer just say you don't know. DON'T make anything up.

            Context: {context}
            """
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriever, 
        "question": RunnablePassthrough(),
        "history": load_memory,
    } 
    | prompt 
    | llm
)

def invoke_chain(question: str) -> str:
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result.content)


invoke_chain("Is Aaronson guilty?")

According to the context provided, Winston believes that Jones, Aaronson, and Rutherford are guilty of the crimes they are charged with, even though he reflects that he had seen a photograph that disproved their guilt. However, he dismisses the memory of the photograph as something he invented, indicating that he has come to accept the Party's version of reality. Therefore, within the narrative, the guilt of Aaronson (and the others) is accepted as a fact by Winston, even if it is known to be false.


In [8]:
invoke_chain("What message did he write in the table?")

Winston wrote "FREEDOM IS SLAVERY" and beneath it, he wrote "TWO AND TWO MAKE FIVE."


In [9]:
invoke_chain("Who is Julia?")

Julia is a character who is Winston's love interest. She represents a rebellious spirit against the Party and is significant in Winston's life as she shares his feelings of discontent and desire for personal freedom. Throughout their relationship, they engage in a romantic affair that symbolizes rebellion against the oppressive regime they live under.


In [10]:
invoke_chain("My name is Gwon. Hallo")

Hello, Gwon! How can I assist you today?


In [11]:
invoke_chain("What's my name?")

Your name is Gwon.


In [12]:
invoke_chain("What message did he write in the table?")

Winston wrote "FREEDOM IS SLAVERY" and beneath it, he wrote "TWO AND TWO MAKE FIVE."
